# Editing the FitnessCalculatorConfiguration: A Step-by-Step Guide

To run this notebook, you need to have an existing FitnessCalculatorConfiguration on Nexus.

Please fill in the values to match the metadata of the FitnessCalculatorConfiguration you want to edit.

In [3]:
emodel = "L5_TPC"
etype = "cAC"
mtype = None
ttype = None
iteration_tag = "XXXX-XX-XX"
species = "mouse"
brain_region = "SSCX"

nexus_project = "" # specify the nexus project to use
nexus_organisation = "" # specify the nexus organisation to use
nexus_endpoint = "" # specify the nexus endpoint to use

# Advanced settings (only change if you know what you are doing)
forge_path = "./forge.yml"
forge_ontology_path = "./nsg.yml"

This next cell will ask for your Nexus token:

In [ ]:
from bluepyemodel.access_point.nexus import NexusAccessPoint

data_access_point = "nexus"
access_point = NexusAccessPoint(
    emodel=emodel,
    species=species,
    brain_region=brain_region,
    project=nexus_project,
    organisation=nexus_organisation,
    endpoint=nexus_endpoint,
    forge_path=forge_path,
    forge_ontology_path=forge_ontology_path,
    etype=etype,
    mtype=mtype,
    ttype=ttype,
    iteration_tag=iteration_tag
)

In [5]:
fitness_config = access_point.get_fitness_calculator_configuration()

print(fitness_config)

Fitness Calculator Configuration:

Protocols:
   {'name': 'IDrest_130', 'stimuli': [{'delay': 700.5, 'amp': 0.4508745591995102, 'thresh_perc': 136.24160437248662, 'duration': 2000.25, 'totduration': 3000.0, 'holding_current': -0.09770833142101765}], 'recordings_from_config': [{'type': 'CompRecording', 'name': 'IDrest_130.soma.v', 'location': 'soma', 'variable': 'v'}], 'validation': False, 'protocol_type': 'ThresholdBasedProtocol', 'stochasticity': False}
   {'name': 'IDrest_200', 'stimuli': [{'delay': 700.5, 'amp': 0.6552015231929614, 'thresh_perc': 197.9834632177735, 'duration': 2000.25, 'totduration': 3000.0, 'holding_current': -0.09765624813735485}], 'recordings_from_config': [{'type': 'CompRecording', 'name': 'IDrest_200.soma.v', 'location': 'soma', 'variable': 'v'}], 'validation': False, 'protocol_type': 'ThresholdBasedProtocol', 'stochasticity': False}
   {'name': 'IV_0', 'stimuli': [{'delay': 20.0, 'amp': 0.00016210680283757512, 'thresh_perc': 0.048984114201291125, 'duration': 1

In [6]:
from bluepyemodel.evaluation.efeature_configuration import EFeatureConfiguration
from bluepyemodel.evaluation.protocol_configuration import ProtocolConfiguration

As an example, we add a new protocol called `Step_1000`

In [7]:
protocol_name = "Step_1000"

stimuli = [{
    "amp": None,
    "delay": 700,
    "duration": 4,
    "holding_current": None,
    "thresh_perc": 1000,
    "totduration": 1000
}]

recordings = [
    {
        "type": "CompRecording",
        "location": "soma",
        "name": f"{protocol_name}.soma.v",
        "variable": "v"
    },
]

protocol_config = ProtocolConfiguration(
    name=protocol_name,
    stimuli=stimuli,
    recordings=recordings,
    validation=False
)

fitness_config.protocols.append(protocol_config)

Note that the name of the protocol matters as BluePyEmodel will use it to detect which shape the protocol should have. For example, here, "step" is in the name and therefore a step protocol will be generated. A complete list of the eCode that can be used to create protocols can be seen [here](https://github.com/BlueBrain/BluePyEModel/blob/main/bluepyemodel/ecode/__init__.py). For more complex eCode, contact the package authors for an explanation of how to use them.§

Next, we add the feature associated with the newly defined protocol.

In [11]:
efeature_def = EFeatureConfiguration(
    efel_feature_name="Spikecount",
    protocol_name=protocol_name,
    recording_name="soma.v",
    mean=1.0,
    std=0.1,
    efel_settings={"strict_stiminterval": False},
    threshold_efeature_std=None,
)

fitness_config.efeatures.append(efeature_def)
print(fitness_config)

Fitness Calculator Configuration:

Protocols:
   {'name': 'IDrest_130', 'stimuli': [{'delay': 700.5, 'amp': 0.4508745591995102, 'thresh_perc': 136.24160437248662, 'duration': 2000.25, 'totduration': 3000.0, 'holding_current': -0.09770833142101765}], 'recordings_from_config': [{'type': 'CompRecording', 'name': 'IDrest_130.soma.v', 'location': 'soma', 'variable': 'v'}], 'validation': False, 'protocol_type': 'ThresholdBasedProtocol', 'stochasticity': False}
   {'name': 'IDrest_200', 'stimuli': [{'delay': 700.5, 'amp': 0.6552015231929614, 'thresh_perc': 197.9834632177735, 'duration': 2000.25, 'totduration': 3000.0, 'holding_current': -0.09765624813735485}], 'recordings_from_config': [{'type': 'CompRecording', 'name': 'IDrest_200.soma.v', 'location': 'soma', 'variable': 'v'}], 'validation': False, 'protocol_type': 'ThresholdBasedProtocol', 'stochasticity': False}
   {'name': 'IV_0', 'stimuli': [{'delay': 20.0, 'amp': 0.00016210680283757512, 'thresh_perc': 0.048984114201291125, 'duration': 1

Finally, we save the edited FitnessCalculatorConfiguration back to Nexus

In [9]:
access_point.store_fitness_calculator_configuration(fitness_config)

<action> _deprecate_one
<succeeded> True
<action> _register_one
<succeeded> True
<action> _update_one
<succeeded> True


To verify that the optimization works with the new FCC, run the following command:

In [ ]:
!python pipeline.py --step=test_optimise --emodel={emodel} --etype={etype} --iteration_tag={iteration_tag} --ttype={ttype}